In [1]:
import pandas as pd
import duckdb
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import pandas_market_calendars as mcal
import holidays
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import stata_setup

working_folder = "D:/FINRA_Working_Data"
monthly_data = "E:/FINRA_Data_by_Month"

In [4]:
# Manually downloaded short sale data from FINRA website 4/28/24
# FNRA - ADF
# FNSQ - Nasdaq Carteret
# FNQC - Nasdaq Chicago
# FNYX - NYSE 

# FINRA Daily Shorting Volume Wrangling
Steps for redoing:
1. Get all daily volume files 
2. Delete 20100915 
3. Clean up data types, consolidate days before 8/1/18
4. Add ShortExemptVolume for days before 2/28/11
5. Combine all days into one file 

In [0]:
# Get daily shorting files from FINRA website (webscraped since API only returns data from past 1 year)


# Set date ranges (consolidated files available from 8/1/18)
start_date_1 = datetime(2010, 1, 1)
end_date_1 = datetime(2018, 7, 31)
start_date_2 = datetime(2018, 8, 1)
end_date_2 = datetime(2024, 8, 9)

# File suffixes
facilities_1 = ["FNRA", "FNSQ", "FNYX"]
facility_2 = "CNMS"

reports_dir = "D:/FINRA_Volume/Raw_Daily_Volume"
# os.makedirs(reports_dir, exist_ok=True)

# Function to query data and save to files with retry logic
def fetch_and_save_data(date_str, facility, retries=5):
    url = f"https://cdn.finra.org/equity/regsho/daily/{facility}shvol{date_str}.txt"
    attempt = 0

    while attempt < retries:
        try:
            # Query the URL with a timeout
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                #cleaned_content = response.text.replace('\n', '')

                filename = f"{date_str}_{facility}.txt"
                file_path = os.path.join(reports_dir, filename)

                with open(file_path, 'w') as file:
                    #file.write(cleaned_content)
                    file.write(response.text)

                return  # Exit the function if successful

            else:
                #print(f"Failed to retrieve data for {date_str} and {facility}: {response.status_code}")
                return  # Exit the function if failure

        except requests.exceptions.RequestException as e:
            print(f"Error occurred for {date_str} and {facility}: {e}")
            attempt += 1
            if attempt == retries:
                print(f"Final retry FAILED... ({attempt}/{retries})")
                time.sleep(2)  # Wait before retrying

# Function to process a range of dates and facilities in parallel
def process_dates_in_parallel(start_date, end_date, facilities):
    with ThreadPoolExecutor(max_workers=10) as executor:
        futures = []
        current_date = start_date

        while current_date <= end_date:
            date_str = current_date.strftime('%Y%m%d')
            for facility in facilities:
                futures.append(executor.submit(fetch_and_save_data, date_str, facility))
            current_date += timedelta(days=1)

        for future in as_completed(futures):
            future.result()

# Process the first date range (2009-2018) in parallel
process_dates_in_parallel(start_date_1, end_date_1, facilities_1)

# Process the second date range (2018-2024) in parallel
process_dates_in_parallel(start_date_2, end_date_2, [facility_2])

print("Script finished running.")

In [35]:
# Clean daily volume files


# Need to define folder_path, which should contain raw files
input_folder = "D:/FINRA_Volume/Raw_Daily_Volume"
output_folder = "D:/FINRA_Volume/Consolidated_Daily"

# Consolidate days before 8/1/18
# Add ShortExemptVolume for days before 2/28/11
            
# Loop over each day in 2010-2023
for year in range(2010, 2024):
    metadata = {}

    for month in range(1, 13):
            
        for day in range(1,32):
            
            date = f"{year}{month:02d}{day:02d}"
    
            if (year < 2018) | ((year == 2018)& (month <= 7)):
                file_list = [file for file in os.listdir(input_folder) if date in file]
        
                # Combine multiple files per day
                if file_list:
                
                    combined_df = pd.DataFrame()
            
                    for file in file_list:
                        df = pd.read_csv(os.path.join(input_folder, file), delimiter='|', index_col=False)
                        df = df.iloc[:-1, :]  # delete check row
                        if (year == 2010) | ((year == 2011) & (month == 1)) | ((year == 2011) & (month == 2) & (day <= 27)):
                            df.insert(df.columns.get_loc('ShortVolume') + 1, 'ShortExemptVolume', 0)
                        if not df.empty and not df.isna().all().all():
                            df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d', errors='coerce')
                            df = df.dropna(subset=['Date', 'Symbol', 'ShortVolume', 'ShortExemptVolume', 'TotalVolume'])
                            combined_df = pd.concat([combined_df, df])
    
                    combined_df = pd.concat([combined_df, df])
            
                    combined_df = combined_df.groupby(['Date', 'Symbol'], as_index=False)[['ShortVolume', 'ShortExemptVolume','TotalVolume']].sum()
                    
                    combined_df = combined_df.sort_values(by=['Symbol']).reset_index(drop=True)
                    combined_df.to_csv(output_folder + '/' + date + '.txt', sep='|', index=False)
            
            else:
                file_path = input_folder + f'/{year}{month:02d}{day:02d}_CNMS.txt'

                if os.path.exists(file_path):
                    df = pd.read_csv(file_path, delimiter='|', index_col=False)
                    df = df.iloc[:-1, :]  # delete check row
                    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d', errors='coerce')
                    df = df.dropna(subset=['Date', 'Symbol', 'ShortVolume', 'ShortExemptVolume', 'TotalVolume'])
                    df.to_csv(output_folder + '/' + f'/{year}{month:02d}{day:02d}.txt', sep='|', index=False)
            print(date)

# Combine daily files into one
dfs = []

# Loop through all text files in the folder
for file in glob.glob(os.path.join(folder_path, '*.txt')):
    df = pd.read_csv(file, sep='|', index_col=False, parse_dates=['Date'], date_format='%Y-%m-%d') 
    dfs.append(df)

combined_df = pd.concat(dfs, ignore_index=True)
combined_df.sort_values(by=['Date', 'Symbol'], inplace=True)

combined_df.to_pickle('D:/FINRA_Volume/daily_volume.pkl')


## FINRA Monthly Detailed Shorting Data Wrangling ##


In [0]:
# Create monthly files from raw files

# Need to define folder_path, which should contain raw files
folder_path = "D:/FINRA_Raw_Data"

# Loop over each month in 2010-2023
for year in range(2010, 2024):
    metadata = {}
    
    for month in range(1, 13):

        year_month = f"{year}{month:02d}"

        file_list = [file for file in os.listdir(folder_path) if year_month in file]
        
        # Combine files 
        combined_df = pd.DataFrame()

        for file in file_list:
            df = pd.read_csv(os.path.join(folder_path, file), delimiter='|', index_col=False, parse_dates=['Date'], date_format='ISO8601')
            df['Time'] = pd.to_datetime(df['Time'], format='%H:%M:%S').dt.time
            df = df.iloc[:-1, :-1]  # delete check row and empty last column
            combined_df = pd.concat([combined_df, df])

        # Sort the DataFrame by symbol, date and time
        combined_df = combined_df.sort_values(by=['Symbol','Date', 'Time']).reset_index(drop=True)

        # Store metadata
        month_metadata = {}
        month_metadata['num_rows'] = combined_df.shape[0]
        month_metadata['unique_market_centers'] = combined_df['MarketCenter'].unique().tolist()
        month_metadata['min_date'] = combined_df['Date'].min()
        month_metadata['max_date'] = combined_df['Date'].max()
        metadata[year_month] = month_metadata

        # Save the concatenated data as a .pkl file
        combined_df.to_pickle(monthly_data + '/' + year_month + '.pkl')
        print(year_month)

for year in range(2010, 2024):# Save metadata
    meta = pd.DataFrame(metadata).T
    meta = meta.rename_axis('Month').reset_index()  # Rename the first column heading as 'Month'
    meta.to_csv(working_folder+'/Metadata_'+f"{year}"+'.txt', sep='|', index=False)

In [8]:
metadata= {}

In [0]:
# Update metadata

month1 = pd.read_pickle(monthly_data+'/202302.pkl')
month1['Date'] = pd.to_datetime(month1['Date'], format='ISO8601')
month1['Time'] = pd.to_datetime(month1['Time'], format='%H:%M:%S').dt.time

metadata['202302'] = {'num_rows': month1.shape[0],
                      'unique_market_centers': month1['MarketCenter'].unique().tolist(),
                      'min_date': month1['Date'].min(),
                      'max_date': month1['Date'].max()}

meta = pd.DataFrame(metadata).T
meta = meta.rename_axis('Month').reset_index()  # Rename the first column heading as 'Month'
meta.to_csv(working_folder+'/Metadata_fillinpartial.txt', sep='|', index=False)

In [0]:
# Create 1-minute or 5-minute aggregated files from monthly files

agg_level = '5min'
#agg_level = '1min'

con = duckdb.connect()

for year in range(2010, 2024):
    for month in range(1, 13):
        #if year == 2010 and month == 1:
            #continue  # Skip the first month of 2010
            
        file_path = f'{monthly_data}/{year}{month:02d}.pkl'

        try:
            raw_daily = pd.read_pickle(file_path)
        except FileNotFoundError:
            print(f'{monthly_data}/{year}{month:02d}.pkl' + ' not found')
            continue  # Skip missing files

        # Round the timestamp down to agg_level
        raw_daily['Time'] = pd.to_datetime(raw_daily['Time'], format='%H:%M:%S')
        raw_daily['Time'] = raw_daily['Time'].dt.floor(agg_level).dt.time

        con.register('raw_daily', raw_daily)

        # Aggregate shorts where ShortType is 'S'
        agg_data_s = con.execute('''
            SELECT Symbol, Date, Time, SUM(Size) AS Shorts
            FROM raw_daily
            WHERE ShortType = 'S'
            GROUP BY Symbol, Date, Time
        ''').fetchdf()

        # Aggregate shorts where ShortType is not 'S'
        agg_data_exempt = con.execute('''
            SELECT Symbol, Date, Time, SUM(Size) AS ExemptShorts
            FROM raw_daily
            WHERE ShortType != 'S'
            GROUP BY Symbol, Date, Time
        ''').fetchdf()

        # Merge the results
        agg_data = con.execute('''
            SELECT 
                a.Symbol, a.Date, a.Time,
                COALESCE(a.Shorts, 0) AS Shorts,
                COALESCE(b.ExemptShorts, 0) AS ExemptShorts
            FROM agg_data_s a
            FULL OUTER JOIN agg_data_exempt b
            ON a.Symbol = b.Symbol AND a.Date = b.Date AND a.Time = b.Time
        ''').fetchdf()

        agg_data.to_pickle(f'{working_folder}/agg_{year}{month:02d}_{agg_level}.pkl')
        #print(f'{year} {month} max exempt shorts: {agg_data["ExemptShorts"].max()}')
        print(f'{year} {month} 5min agg done')

con.close()

In [0]:
# Create coarser aggregated files from 5-minute aggregated files
# Daily, 1 hour or 30 min

for year in range(2011, 2024):
    annual_agg_data = pd.DataFrame()  # Initialize a new DataFrame for each year

    for month in range(1, 13):

        file_path = f'{working_folder}/FINRA_5min_Agg/agg_{year}{month:02d}_5min.pkl'

        try:
            monthly_data = pd.read_pickle(file_path)
        except FileNotFoundError:
            print(f'{file_path} not found')
            continue  # Skip missing files

        # monthly_data['Hour'] = pd.to_datetime(monthly_data['Time'], format='%H:%M:%S').dt.hour
        #monthly_data['HalfHour'] = pd.to_datetime(monthly_data['Time'], format='%H:%M:%S').dt.floor('30min').dt.time

        # Aggregate trades and sort
        monthly_data = monthly_data.groupby(['Symbol', 'Date']).agg({ 'Shorts': 'sum', 'ExemptShorts': 'sum'}).reset_index()
        monthly_data = monthly_data.sort_values(by=['Symbol', 'Date']).reset_index(drop=True)
        # monthly_data = monthly_data.groupby(['Symbol', 'Date', 'Hour']).agg({ 'Shorts': 'sum', 'ExemptShorts': 'sum'}).reset_index()
        # monthly_data = monthly_data.sort_values(by=['Symbol', 'Date', 'Hour']).reset_index(drop=True)
        # monthly_data = monthly_data.groupby(['Symbol', 'Date', 'HalfHour']).agg({ 'Shorts': 'sum', 'ExemptShorts': 'sum'}).reset_index()
        # monthly_data = monthly_data.sort_values(by=['Symbol', 'Date', 'HalfHour']).reset_index(drop=True)

        # Append the monthly aggregated data to the annual DataFrame
        annual_agg_data = pd.concat([annual_agg_data, monthly_data], ignore_index=True)

    # Save the aggregated data for the entire year
    annual_agg_data.to_pickle(f'D:/FINRA_Daily_Agg/agg_{year}.pkl')
    print(f'{year} data aggregated by day')
    # annual_agg_data.to_pickle(f'{working_folder}/FINRA_1hr_Agg/agg_{year}_1hr.pkl')
    # print(f'{year} data aggregated by hour')
    # annual_agg_data.to_pickle(f'{working_folder}/FINRA_30min_Agg/agg_{year}_30min.pkl')
    # print(f'{year} data aggregated by half hour')


# IBES Data Cleaning + Create Table of EAs and Relevant Trading Dates


In [5]:
# Clean the I/B/E/S data downloaded from WRDS 08/23/2024
# I/B/E/S Unadjusted Surprise History for fiscal periods ending 2009-01 to 2024-01 (only has IBES ticker and official ticker)


surprise = pd.read_csv(working_folder + '/statsum_unadjsurp_original.csv', delimiter = ',')
surprise = surprise.drop(columns=['MEASURE', 'FISCALP', 'PYEAR', 'PMON', 'actual'])
surprise = surprise.dropna()

# Drop OFTIC-anndats duplicates
duplicates = surprise.duplicated(subset=['TICKER', 'anndats'], keep=False)
surprise = surprise[~duplicates]

# Clean IBES unadjusted actuals
actuals = pd.read_csv(working_folder + '/actual_unadj_original.csv', delimiter = ',')
actuals = actuals.drop(columns=['MEASURE', 'PDICITY'])
actuals = actuals.dropna()

# Combine date and time columns into a single datetime column
actuals['ANNOUNCE'] = pd.to_datetime(actuals['ANNDATS'] + ' ' + actuals['ANNTIMS'], format='%Y-%m-%d %H:%M:%S')
actuals['ACTIVATE'] = pd.to_datetime(actuals['ACTDATS'] + ' ' + actuals['ACTTIMS'], format='%Y-%m-%d %H:%M:%S')
actuals['PENDS'], actuals['ACTDATS'], actuals['ACTTIMS'] = pd.to_datetime(actuals['PENDS'], format='%Y-%m-%d'), pd.to_datetime(actuals['ACTDATS'], format='%Y-%m-%d'), pd.to_datetime(actuals['ACTTIMS'], format='%H:%M:%S')

# Keep the row with the latest activation for a given CUSIP-PENDS combination
actuals = actuals.sort_values(by=['CUSIP', 'PENDS', 'ACTIVATE'], ascending=[True, True, True]).groupby(['CUSIP', 'PENDS']).last().reset_index()

# Drop rows that have earnings announcements earlier than the fiscal period end
actuals = actuals[actuals['ANNOUNCE'].dt.date >=actuals['PENDS']]

# Merge firm identifiers, actual EPS, and announcement time from IBES unadjusted actuals
con = duckdb.connect()
con.register('surprise', surprise)
con.register('actuals', actuals)
surprise = con.execute("""
    SELECT 
        surprise.*, 
        actuals.CUSIP, actuals.PENDS , actuals.CNAME , actuals.ANNTIMS, actuals.VALUE
    FROM 
        surprise 
    LEFT JOIN (
        SELECT 
            TICKER, CUSIP, PENDS, CNAME, ANNDATS, ANNTIMS, VALUE
        FROM actuals
    ) AS actuals
    ON 
        surprise.TICKER = actuals.TICKER
        AND surprise.anndats = actuals.ANNDATS
    ORDER BY 
        surprise.OFTIC, 
        surprise.anndats
""").df()
con.close()

surprise['ANNOUNCE'] = pd.to_datetime(surprise['anndats'] + ' ' + surprise['ANNTIMS'], format='%Y-%m-%d %H:%M:%S')
surprise['ANNOUNCE'] = surprise['ANNOUNCE'].dt.floor('5min')

# Keep announcements Jan. 5 2010 - Dec. 28 2023 
surprise = surprise[(surprise['ANNOUNCE'] >= '2010-01-05') & (surprise['ANNOUNCE'] <= '2023-12-28')]

# Calculate surprise
surprise['SURPRISE'] = surprise['VALUE'] - surprise['surpmean']

surprise = surprise.sort_values(by=['OFTIC', 'ANNOUNCE'], ascending=[True, True]).reset_index(drop=True)
surprise = surprise.drop(columns=['anndats', 'ANNTIMS', 'TICKER'])
surprise = surprise[['CUSIP', 'OFTIC', 'CNAME', 'PENDS', 'ANNOUNCE', 'SURPRISE']] # Drop and reorder columns
surprise.to_csv(working_folder + '/statsum_unadjsurp_clean.csv', index=False)

In [6]:
ea_dates = pd.read_csv(working_folder + '/statsum_unadjsurp_clean.csv', parse_dates=['PENDS', 'ANNOUNCE'], date_format='ISO8601')
ea_dates = ea_dates[['CUSIP', 'OFTIC', 'CNAME', 'ANNOUNCE', 'SURPRISE']]
ea_dates['ANNOUNCE'] = pd.to_datetime(ea_dates['ANNOUNCE']).dt.tz_localize(None)

# Identify positive and negative surprises
ea_dates['POS_SURPRISE'] = ea_dates['SURPRISE'] > 0

# Identify pre-open and post-close announcements
ea_dates['pre_open'] = False
ea_dates['post_close'] = False

announce_time = ea_dates['ANNOUNCE'].dt.time

ea_dates.loc[announce_time < pd.to_datetime('09:30:00').time(), 'pre_open'] = True
ea_dates.loc[announce_time >= pd.to_datetime('16:00:00').time(), 'post_close'] = True

# Drop EAs within trading hours
ea_dates = ea_dates[ea_dates['pre_open'] | ea_dates['post_close']]

# Define trading calendar
nyse_holidays = holidays.NYSE()

# Function to get previous and next trading days
def get_surr_trading_days(date):

    one_day = pd.Timedelta(days=1)

    def get_previous_trading_day(day, offset=1):
        for _ in range(offset):
            day -= one_day
            while day.weekday() >= 5 or day in nyse_holidays: # Skip weekends and holidays
                day -= one_day
        return day

    def get_next_trading_day(day, offset=1):
        for _ in range(offset):
            day += one_day
            while day.weekday() >= 5 or day in nyse_holidays: # Skip weekends and holidays
                day += one_day
        return day

    # post-close announcements
    if date.time() >= pd.to_datetime('12:00:00').time():
        day_before = date
        while day_before.weekday() >= 5 or day_before in nyse_holidays:
            day_before -= one_day
        day_after = get_next_trading_day(day_before)

    # pre-open announcements
    else:
        day_before = get_previous_trading_day(date)

        day_after = date
        while day_after.weekday() >= 5 or day_after in nyse_holidays:
            day_after += one_day

    # Calculate the previous days
    day_2_before = get_previous_trading_day(day_before)
    day_3_before = get_previous_trading_day(day_2_before)
    day_4_before = get_previous_trading_day(day_3_before)
    day_5_before = get_previous_trading_day(day_4_before)
    day_6_before = get_previous_trading_day(day_5_before)
    day_25_before = get_previous_trading_day(day_6_before, offset=19)

    day_2_after = get_next_trading_day(day_after)
    day_3_after = get_next_trading_day(day_2_after)
    day_4_after = get_next_trading_day(day_3_after)
    day_5_after = get_next_trading_day(day_4_after)
    
    return  day_5_after.date(), day_4_after.date(), day_3_after.date(), day_2_after.date(), day_after.date(), day_before.date(), day_2_before.date(), day_3_before.date(), day_4_before.date(), day_5_before.date(), day_6_before.date(), day_25_before.date()

ea_dates['day_5_after'], ea_dates['day_4_after'], ea_dates['day_3_after'], ea_dates['day_2_after'], ea_dates['day_after'], ea_dates['day_before'], ea_dates['day_2_before'], ea_dates['day_3_before'], ea_dates['day_4_before'], ea_dates['day_5_before'], ea_dates['day_6_before'], ea_dates['day_25_before'] = zip(*ea_dates['ANNOUNCE'].apply(lambda x: get_surr_trading_days(x)))

In [7]:
# Add point-in-tickers, matching on CUSIP using linking table from Algoseek (thanks Matt)

link = pd.read_parquet(working_folder + '/crsp_aseek.parquet')

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('ea_dates', ea_dates)
con.register('link', link)

query = """
SELECT ea.CUSIP, lk.permno, lk.permco, lk.ticker as pit_ticker, ea.OFTIC, ea.CNAME, ea.ANNOUNCE, ea.SURPRISE, ea.POS_SURPRISE, ea.pre_open, ea.post_close, ea.day_5_after, ea.day_4_after, ea.day_3_after, ea.day_2_after, ea.day_after, ea.day_before, ea.day_2_before, ea.day_3_before, ea.day_4_before, ea.day_5_before, ea.day_6_before, ea.day_25_before
    FROM ea_dates ea
    LEFT JOIN (
        SELECT cusip, bdate, edate, permno, permco, ticker
        FROM link
    ) lk
    ON ea.CUSIP = substr(lk.cusip, 1, 8)
    WHERE ea.ANNOUNCE BETWEEN lk.bdate AND lk.edate 
    ORDER BY ea.ANNOUNCE, lk.ticker
"""

# Execute the query 
ea_dates = con.execute(query).df()
con.close()

ea_dates.to_csv(working_folder+ '/ea_dates.csv', index = False)

In [8]:
ea_dates

# Quick analysis
# ea_dates.shape # 192416
# ea_dates['pre_open'].sum() # 86646; 45.0% are pre-open
# ea_dates['post_close'].sum() # 105770; 55.0% are post_close

,CUSIP,permno,permco,pit_ticker,OFTIC,CNAME,ANNOUNCE,SURPRISE,POS_SURPRISE,pre_open,...,day_3_after,day_2_after,day_after,day_before,day_2_before,day_3_before,day_4_before,day_5_before,day_6_before,day_25_before
0,83545110,76568,10843,SONC,SONC,SONIC CORP,2010-01-05 16:00:00,-0.03368,False,False,...,2010-01-08,2010-01-07,2010-01-06,2010-01-05,2010-01-04,2009-12-31,2009-12-30,2009-12-29,2009-12-28,2009-11-30
1,03475V10,90179,45192,ANGO,ANGO,ANGIODYNAMICS,2010-01-05 16:05:00,0.02000,True,False,...,2010-01-08,2010-01-07,2010-01-06,2010-01-05,2010-01-04,2009-12-31,2009-12-30,2009-12-29,2009-12-28,2009-11-30
2,51476610,83148,14374,LNDC,LNDC,LANDEC CORP,2010-01-05 16:05:00,-0.00333,False,False,...,2010-01-08,2010-01-07,2010-01-06,2010-01-05,2010-01-04,2009-12-31,2009-12-30,2009-12-29,2009-12-28,2009-11-30
3,87815510,65752,4402,TISI,TISI,TEAM INC,2010-01-05 16:05:00,0.04250,True,False,...,2010-01-08,2010-01-07,2010-01-06,2010-01-05,2010-01-04,2009-12-31,2009-12-30,2009-12-29,2009-12-28,2009-11-30
4,61945A10,90386,45695,MOS,MOS,MOSAIC CO/THE,2010-01-05 16:15:00,-0.03176,False,False,...,2010-01-08,2010-01-07,2010-01-06,2010-01-05,2010-01-04,2009-12-31,2009-12-30,2009-12-29,2009-12-28,2009-11-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192411,69012T20,16104,55569,OTLK,OTLK,OUTLOOK,2023-12-22 08:05:00,0.00875,True,True,...,2023-12-27,2023-12-26,2023-12-22,2023-12-21,2023-12-20,2023-12-19,2023-12-18,2023-12-15,2023-12-14,2023-11-16
192412,12482W30,17091,56139,YCBD,YCBD,CBDMD,2023-12-22 16:00:00,-0.45000,False,False,...,2023-12-28,2023-12-27,2023-12-26,2023-12-22,2023-12-21,2023-12-20,2023-12-19,2023-12-18,2023-12-15,2023-11-17
192413,16842Q10,16883,56045,CSSE,CSSE,CHICKEN SOUP,2023-12-22 17:00:00,-0.68600,False,False,...,2023-12-28,2023-12-27,2023-12-26,2023-12-22,2023-12-21,2023-12-20,2023-12-19,2023-12-18,2023-12-15,2023-11-17
192414,L7756P10,21955,58741,PROC,PROC,PROCAPS GROUP,2023-12-26 17:25:00,0.03000,True,False,...,2023-12-29,2023-12-28,2023-12-27,2023-12-26,2023-12-22,2023-12-21,2023-12-20,2023-12-19,2023-12-18,2023-11-20


# CRSP Data Cleaning + Merge EA Dates w/ CRSP

In [102]:
# Get list of unique CUSIPs from IBES quarterly announcements 
# Used for downloading less data from WRDS


eps = pd.read_csv(working_folder + '/statsum_unadjsurp_clean.csv', delimiter = ',')

# Extract unique values from the 'CUSIP' column
unique_cusip = eps['CUSIP'].unique()

# Save the unique values to a .txt file
with open('D:/FINRA_Working_Data/unique_cusip.txt', 'w') as file:
    for cusip in unique_cusip:

        file.write(f"{cusip}\n")

In [90]:
# Clean the CRSP data downloaded from WRDS 8/15/24
# CRSP Daily Stock file for 2009-2023, only for CUSIPs from cleaned IBES quarterly EAs


# Specify the correct data types for the problematic columns
dtype_spec = {
    'PERMNO': 'str',
    'SICCD': 'str',
    'NCUSIP': 'str',
    'SHRCLS': 'str',
    'PERMCO': 'str',
    'CUSIP': 'str',
    'RET': 'str',
    'RETX': 'str'
}

# Import the data with specified data types
crsp = pd.read_csv(working_folder + '/crsp_daily_original.csv', delimiter = ',', dtype=dtype_spec, parse_dates=['date'], date_format='%Y-%m-%d')
crsp['date'] = pd.to_datetime(crsp['date']).dt.tz_localize(None)

# Fix column data types
for col in ['SICCD', 'SHRCD', 'NAICS']: # to int
    crsp[col] = pd.to_numeric(crsp[col], errors='coerce') # replace values that can't be converted with NaN
    crsp = crsp.dropna(subset=[col])
    crsp[col] = crsp[col].astype(int)

for col in ['RET', 'RETX']: # to float
    crsp[col] = pd.to_numeric(crsp[col], errors='coerce') # replace values that can't be converted with NaN
    crsp = crsp.dropna(subset=[col])
    crsp[col] = crsp[col].astype(float)

# Adjust prices and number of shares
crsp = crsp.dropna(subset=['CFACPR', 'CFACSHR'])
crsp = crsp[crsp['CFACPR'] != 0]
crsp['PRC'] = crsp['PRC'].abs()
crsp['PRC_scaled'] = crsp['PRC']/crsp['CFACPR']
crsp['SHROUT'] = crsp['SHROUT']*crsp['CFACSHR']

# Drop thinly traded stocks (<$5) based on PRC, not PRC_scaled

# Drop share class column and missing values of volume
crsp = crsp.drop(columns=['SHRCLS'])
crsp = crsp.dropna(subset=['VOL'])
crsp = crsp[crsp['VOL']>0]

crsp = crsp.drop(columns=['BID', 'ASK', 'BIDLO', 'ASKHI', 'NUMTRD'])

crsp = crsp[~crsp['RET'].isin([-66, -77, -88, -99])].reset_index(drop=True)
crsp = crsp.sort_values(by=['PERMNO', 'date'])
crsp['LRET1'] = crsp.groupby('PERMNO')['RET'].shift(1)
crsp['LRET2'] = crsp.groupby('PERMNO')['RET'].shift(2)
crsp['LRET3'] = crsp.groupby('PERMNO')['RET'].shift(3)
crsp['LRET4'] = crsp.groupby('PERMNO')['RET'].shift(4)
crsp['LRET5'] = crsp.groupby('PERMNO')['RET'].shift(5)
crsp['FRET1'] = crsp.groupby('PERMNO')['RET'].shift(-1)
crsp['FRET2'] = crsp.groupby('PERMNO')['RET'].shift(-2)
crsp['FRET3'] = crsp.groupby('PERMNO')['RET'].shift(-3)
crsp['FRET4'] = crsp.groupby('PERMNO')['RET'].shift(-4)
crsp['FRET5'] = crsp.groupby('PERMNO')['RET'].shift(-5)
crsp['RET1_2'] = crsp['RET'] + crsp['FRET1']
crsp['RET1_3'] = crsp['RET'] + crsp['FRET1'] + crsp['FRET2']

crsp = crsp.dropna()
crsp.to_csv(working_folder + '/crsp_daily_clean.csv', index=False)

In [15]:
crsp.head()

,PERMNO,date,SHRCD,SICCD,NCUSIP,TICKER,COMNAM,NAICS,PERMCO,CUSIP,...,LRET3,LRET4,LRET5,FRET1,FRET2,FRET3,FRET4,FRET5,RET1_2,RET1_3
0,10001,2009-01-09,11,4920,29274A20,EWST,ENERGY WEST INC,221210,7953,36720410,...,0.012983,0.013301,0.001211,-0.017321,0.009389,-0.003497,0.038732,-0.039548,0.019805,0.029194
1,10001,2009-01-12,11,4920,29274A20,EWST,ENERGY WEST INC,221210,7953,36720410,...,-0.009283,0.012983,0.013301,0.009389,-0.003497,0.038732,-0.039548,0.005294,-0.007932,-0.011429
2,10001,2009-01-13,11,4920,29274A20,EWST,ENERGY WEST INC,221210,7953,36720410,...,-0.007134,-0.009283,0.012983,-0.003497,0.038732,-0.039548,0.005294,-0.020948,0.005892,0.044624
3,10001,2009-01-14,11,4920,29274A20,EWST,ENERGY WEST INC,221210,7953,36720410,...,0.037126,-0.007134,-0.009283,0.038732,-0.039548,0.005294,-0.020948,0.074576,0.035235,-0.004313
4,10001,2009-01-15,11,4920,29274A20,EWST,ENERGY WEST INC,221210,7953,36720410,...,-0.017321,0.037126,-0.007134,-0.039548,0.005294,-0.020948,0.074576,-0.054494,-0.000816,0.004478


In [9]:
# Merge IBES quarterly announcements with CRSP (on IBES CUSIP and CRSP NCUSIP)


eps = pd.read_csv(working_folder + '/ea_dates.csv', delimiter = ',', parse_dates=['ANNOUNCE', 'day_after', 'day_2_after', 'day_3_after', 'day_4_after', 'day_5_after', 'day_before', 'day_2_before', 'day_3_before', 'day_4_before', 'day_5_before', 'day_6_before', 'day_25_before'], date_format='ISO8601')
crsp = pd.read_csv(working_folder + '/crsp_daily_clean.csv', delimiter = ',', parse_dates=['date'], date_format='%Y-%m-%d', low_memory=False)

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('eps', eps)
con.register('crsp', crsp)



query = """
SELECT 
    eps.*, 
    crsp1.PRC_scaled AS price_6day_lag,
    crsp2.CFACPR AS adjust_day_after,
    crsp2.LRET5 as lret5,
    crsp2.LRET4 as lret4,
    crsp2.LRET3 as lret3,
    crsp2.LRET2 as lret2,
    crsp2.LRET1 as lret1,
    crsp2.RET as ret,
    crsp2.FRET1 as fret1,
    crsp2.FRET2 as fret2,
    crsp2.FRET3 as fret3,
    crsp2.FRET4 as fret4,
    crsp2.RET1_2 as ret1_2,
    crsp2.RET1_3 as ret1_3,
    COALESCE(crsp3.VOL,0) as day5_after_volume_crsp,
    COALESCE(crsp4.VOL,0) as day4_after_volume_crsp,
    COALESCE(crsp5.VOL,0) as day3_after_volume_crsp,
    COALESCE(crsp6.VOL,0) as day2_before_volume_crsp,
    COALESCE(crsp7.VOL,0) as day_before_volume_crsp,
    COALESCE(crsp2.VOL,0) as day_after_volume_crsp,
    COALESCE(crsp8.VOL,0) as day2_after_volume_crsp,
    COALESCE(crsp9.VOL,0) as day3_before_volume_crsp,
    COALESCE(crsp10.VOL,0) as day4_before_volume_crsp,
    COALESCE(crsp11.VOL,0) as day5_before_volume_crsp,
    COALESCE(crsp12.VOL,0) as day6_before_volume_crsp
FROM 
    eps
LEFT JOIN 
    crsp crsp1
ON 
    eps.CUSIP = crsp1.NCUSIP 
    AND eps.day_6_before = crsp1.date
LEFT JOIN 
    crsp crsp2
ON 
    eps.CUSIP = crsp2.NCUSIP 
    AND eps.day_after = crsp2.date
LEFT JOIN 
    crsp crsp3
ON 
    eps.CUSIP = crsp3.NCUSIP 
    AND eps.day_5_after = crsp3.date
LEFT JOIN 
    crsp crsp4
ON 
    eps.CUSIP = crsp4.NCUSIP 
    AND eps.day_4_after = crsp4.date
LEFT JOIN 
    crsp crsp5
ON 
    eps.CUSIP = crsp5.NCUSIP 
    AND eps.day_3_after = crsp5.date
LEFT JOIN 
    crsp crsp6
ON 
    eps.CUSIP = crsp6.NCUSIP 
    AND eps.day_2_before = crsp6.date
LEFT JOIN 
    crsp crsp7
ON 
    eps.CUSIP = crsp7.NCUSIP 
    AND eps.day_before = crsp7.date
LEFT JOIN 
    crsp crsp8
ON 
    eps.CUSIP = crsp8.NCUSIP 
    AND eps.day_2_after = crsp8.date
LEFT JOIN 
    crsp crsp9
ON 
    eps.CUSIP = crsp9.NCUSIP 
    AND eps.day_3_before = crsp9.date
LEFT JOIN 
    crsp crsp10
ON 
    eps.CUSIP = crsp10.NCUSIP 
    AND eps.day_4_before = crsp10.date
LEFT JOIN 
    crsp crsp11
ON 
    eps.CUSIP = crsp11.NCUSIP 
    AND eps.day_5_before = crsp11.date
LEFT JOIN 
    crsp crsp12
ON 
    eps.CUSIP = crsp12.NCUSIP 
    AND eps.day_6_before = crsp12.date
WHERE 
    crsp1.PRC_scaled IS NOT NULL 
    AND crsp2.CFACPR IS NOT NULL
    AND (crsp7.VOL IS NOT NULL
    OR crsp2.VOL IS NOT NULL)
ORDER BY 
    eps.ANNOUNCE, eps.pit_ticker
"""

# Execute the query 
eps_merge = con.execute(query).df()

# Close the connection
con.close()

# Scale earnings surprise by closing price one week before the announcement
# Surprise and price are scaled by the corresponding day's adjustment factor
eps_merge['SURPRISE'] = eps_merge['SURPRISE']/eps_merge['adjust_day_after']
eps_merge['SURPRISE'] = eps_merge['SURPRISE']/eps_merge['price_6day_lag']


eps_merge.to_csv(working_folder + '/ibes_crsp_merge.csv', index=False)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [14]:
eps_merge.head()

,CUSIP,permno,permco,pit_ticker,OFTIC,CNAME,ANNOUNCE,SURPRISE,POS_SURPRISE,pre_open,...,day4_after_volume_crsp,day3_after_volume_crsp,day2_before_volume_crsp,day_before_volume_crsp,day_after_volume_crsp,day2_after_volume_crsp,day3_before_volume_crsp,day4_before_volume_crsp,day5_before_volume_crsp,day6_before_volume_crsp
0,83545110,76568,10843,SONC,SONC,SONIC CORP,2010-01-05 16:00:00,-0.003198,False,False,...,1355876.0,1411107.0,759771.0,888134.0,6754285.0,2032954.0,864570.0,437752.0,215709.0,248900.0
1,03475V10,90179,45192,ANGO,ANGO,ANGIODYNAMICS,2010-01-05 16:05:00,0.001271,True,False,...,144964.0,322411.0,322350.0,157386.0,975932.0,281087.0,63337.0,81643.0,62245.0,62007.0
2,51476610,83148,14374,LNDC,LNDC,LANDEC CORP,2010-01-05 16:05:00,-0.000556,False,False,...,39801.0,39503.0,71586.0,64962.0,180997.0,135191.0,102517.0,107020.0,205939.0,77058.0
3,87815510,65752,4402,TISI,TISI,TEAM INC,2010-01-05 16:05:00,0.002276,True,False,...,102107.0,53627.0,53261.0,117312.0,106846.0,143918.0,53621.0,63789.0,73291.0,48777.0
4,61945A10,90386,45695,MOS,MOS,MOSAIC CO/THE,2010-01-05 16:15:00,-0.000522,False,False,...,4031600.0,4074700.0,4072600.0,6505800.0,9999900.0,5544000.0,1821200.0,1871500.0,3227500.0,2311700.0


In [12]:
# Prep IBES/CRSP merged data to check ERCs


reg_test = pd.read_csv(working_folder + '/ibes_crsp_merge.csv', delimiter = ',', parse_dates=['ANNOUNCE'], date_format='ISO8601')

reg_test.loc[:, 'year'] = reg_test['ANNOUNCE'].dt.year
reg_test.loc[:, 'quarter'] = reg_test['ANNOUNCE'].dt.quarter
reg_test.loc[:,'year_quarter'] = (reg_test['year'] - 2010) * 4 + reg_test['quarter']

# Drop duplicates, keeping the later EA
reg_test = reg_test.sort_values(by=['CUSIP', 'year_quarter', 'ANNOUNCE'])
reg_test = reg_test.drop_duplicates(subset=['CUSIP', 'year_quarter'], keep='last')

reg_test.to_stata(working_folder + '/reg_test.dta')

,CUSIP,permno,permco,pit_ticker,OFTIC,CNAME,ANNOUNCE,SURPRISE,POS_SURPRISE,pre_open,...,day_before_volume_crsp,day_after_volume_crsp,day2_after_volume_crsp,day3_before_volume_crsp,day4_before_volume_crsp,day5_before_volume_crsp,day6_before_volume_crsp,year,quarter,year_quarter
66975,00030710,14945,55038,AAC,AAC,AAC HOLDINGS,2015-02-24 16:05:00,0.003519,True,False,...,159273.0,174032.0,150677.0,94039.0,9343.0,35211.0,18925.0,2015,1,21
69313,00030710,14945,55038,AAC,AAC,AAC HOLDINGS,2015-04-29 16:05:00,0.001504,True,False,...,205740.0,721994.0,182544.0,103761.0,99156.0,190891.0,71138.0,2015,2,22
72754,00030710,14945,55038,AAC,AAC,AAC HOLDG,2015-07-29 16:50:00,0.004313,True,False,...,214241.0,1065299.0,525575.0,372503.0,256674.0,253017.0,330927.0,2015,3,23
75894,00030710,14945,55038,AAC,AAC,AAC HOLDG,2015-10-27 17:15:00,0.002426,True,False,...,264274.0,659659.0,269322.0,122660.0,410267.0,204418.0,193906.0,2015,4,24
80460,00030710,14945,55038,AAC,AAC,AAC HOLDG,2016-02-23 06:30:00,-0.000298,False,True,...,378930.0,572062.0,185404.0,239184.0,320303.0,211663.0,271571.0,2016,1,25
83871,00030710,14945,55038,AAC,AAC,AAC HOLDG,2016-05-05 06:30:00,0.002542,True,True,...,112262.0,464244.0,148874.0,152641.0,137904.0,210377.0,88539.0,2016,2,26
87205,00030710,14945,55038,AAC,AAC,AAC HOLDG,2016-08-04 06:30:00,-0.001913,False,True,...,232277.0,304968.0,270909.0,181192.0,67323.0,117135.0,185273.0,2016,3,27
90392,00030710,14945,55038,AAC,AAC,AAC HOLDG,2016-11-03 06:30:00,-0.002134,False,True,...,246999.0,3742602.0,1246354.0,153957.0,95531.0,87061.0,161290.0,2016,4,28
94219,00030710,14945,55038,AAC,AAC,AAC HOLDG,2017-02-27 18:50:00,-0.000617,False,False,...,173443.0,418117.0,307752.0,165743.0,87871.0,84583.0,84704.0,2017,1,29
96921,00030710,14945,55038,AAC,AAC,AAC HOLDG,2017-05-03 18:35:00,0.009346,True,False,...,104717.0,341148.0,273381.0,51342.0,58351.0,72729.0,121080.0,2017,2,30


# Merge EAs with Shorts

In [16]:
# Gather shorting volume around EAs 


reg_data = pd.read_csv(working_folder + '/ibes_crsp_merge.csv', delimiter = ',', parse_dates=['ANNOUNCE', 'day_after', 'day_before', 'day_2_before', 'day_3_before', 'day_4_before', 'day_5_before', 'day_6_before', 'day_25_before'], date_format='ISO8601')

for year in range(2010, 2024):
    file_path = f'D:/FINRA_Daily_Agg/agg_{year}.pkl'

    # Read the data into a dataframe with the same name as the file
    globals()[f'agg_{year}'] = pd.read_pickle(file_path)

In [17]:
# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

# Register the DataFrame with the DuckDB instance
con.register('reg_data', reg_data)

# Merge reg_data with agg_ files
query = """
WITH short_data AS (
    SELECT Symbol, Date, Shorts
    FROM agg_2010 UNION ALL
    SELECT Symbol, Date, Shorts
    FROM agg_2011 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2012 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2013 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2014 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2015 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2016 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2017 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2018 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2019 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2020 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2021 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2022 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2023
)

SELECT 
    reg_data.*, 
    COALESCE(short_data_day5_before.Shorts, 0) AS day5_be_shorts,
    COALESCE(short_data_day4_before.Shorts, 0) AS day4_be_shorts,
    COALESCE(short_data_day3_before.Shorts, 0) AS day3_be_shorts,
    COALESCE(short_data_day2_before.Shorts, 0) AS day2_be_shorts,
    COALESCE(short_data_day_before.Shorts, 0) AS day_be_shorts,
    COALESCE(short_data_day_after.Shorts, 0) AS day_af_shorts,
    COALESCE(short_data_day2_after.Shorts, 0) AS day2_af_shorts,
    COALESCE(short_data_day3_after.Shorts, 0) AS day3_af_shorts,
    COALESCE(short_data_day4_after.Shorts, 0) AS day4_af_shorts,
    COALESCE(short_data_day5_after.Shorts, 0) AS day5_af_shorts
FROM
    reg_data
LEFT JOIN short_data AS short_data_day5_before
    ON reg_data.pit_ticker = short_data_day5_before.Symbol 
    AND reg_data.day_5_before = short_data_day5_before.Date
LEFT JOIN short_data AS short_data_day4_before
    ON reg_data.pit_ticker = short_data_day4_before.Symbol 
    AND reg_data.day_4_before = short_data_day4_before.Date
LEFT JOIN short_data AS short_data_day3_before
    ON reg_data.pit_ticker = short_data_day3_before.Symbol 
    AND reg_data.day_3_before = short_data_day3_before.Date
LEFT JOIN short_data AS short_data_day2_before
    ON reg_data.pit_ticker = short_data_day2_before.Symbol 
    AND reg_data.day_2_before = short_data_day2_before.Date
LEFT JOIN short_data AS short_data_day_before
    ON reg_data.pit_ticker = short_data_day_before.Symbol 
    AND reg_data.day_before = short_data_day_before.Date
LEFT JOIN short_data AS short_data_day_after
    ON reg_data.pit_ticker = short_data_day_after.Symbol 
    AND reg_data.day_after = short_data_day_after.Date
LEFT JOIN short_data AS short_data_day2_after
    ON reg_data.pit_ticker = short_data_day2_after.Symbol 
    AND reg_data.day_2_after = short_data_day2_after.Date
LEFT JOIN short_data AS short_data_day3_after
    ON reg_data.pit_ticker = short_data_day3_after.Symbol 
    AND reg_data.day_3_after = short_data_day3_after.Date
LEFT JOIN short_data AS short_data_day4_after
    ON reg_data.pit_ticker = short_data_day4_after.Symbol 
    AND reg_data.day_4_after = short_data_day4_after.Date
LEFT JOIN short_data AS short_data_day5_after
    ON reg_data.pit_ticker = short_data_day5_after.Symbol 
    AND reg_data.day_5_after = short_data_day5_after.Date
WHERE 
    short_data_day_before.Shorts IS NOT NULL 
    OR short_data_day_after.Shorts IS NOT NULL
ORDER BY 
    reg_data.ANNOUNCE, 
    reg_data.pit_ticker
"""

# Execute the query 
reg_data = con.execute(query).df()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [18]:
# Average shorting one month prior


# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('reg_data', reg_data)

# Merge reg_data with agg_ files
query = """
WITH short_data AS (
    SELECT Symbol, Date, Shorts
    FROM agg_2010 UNION ALL
    SELECT Symbol, Date, Shorts
    FROM agg_2011 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2012 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2013 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2014 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2015 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2016 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2017 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2018 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2019 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2020 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2021 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2022 UNION ALL 
    SELECT Symbol, Date, Shorts
    FROM agg_2023
)

SELECT 
    reg_data.pit_ticker, reg_data.ANNOUNCE,
    SUM(COALESCE(short_data.Shorts, 0)) / 20 AS avg_month_before_shorts
FROM
    reg_data
LEFT JOIN short_data
    ON reg_data.pit_ticker = short_data.Symbol 
    AND short_data.Date >= reg_data.day_25_before 
    AND short_data.Date <= reg_data.day_6_before
GROUP BY
    reg_data.pit_ticker,
    reg_data.ANNOUNCE
ORDER BY 
    reg_data.pit_ticker,
    reg_data.ANNOUNCE
"""

# Execute the query 
reg_data_avg_short = con.execute(query).df()

# Merge reg_data with agg_ files
query = """
SELECT 
    reg_data.*, reg_data_avg_short.avg_month_before_shorts
FROM
    reg_data
LEFT JOIN reg_data_avg_short
    ON reg_data.pit_ticker = reg_data_avg_short.pit_ticker 
    AND reg_data.ANNOUNCE = reg_data_avg_short.ANNOUNCE 
WHERE 
    reg_data_avg_short.avg_month_before_shorts IS NOT NULL
ORDER BY 
    reg_data.pit_ticker,
    reg_data.ANNOUNCE
"""

# Execute the query 
reg_data = con.execute(query).df()

# Close the connection
con.close()

In [19]:
volume = pd.read_pickle('D:/FINRA_Volume/daily_volume.pkl')

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('reg_data', reg_data)
con.register('volume', volume)

# Merge volume with regression data
query = """
SELECT 
    reg_data.*,
    COALESCE(volume_day6_before.TotalVolume, 0) AS day6_before_volume,
    COALESCE(volume_day5_before.TotalVolume, 0) AS day5_before_volume,
    COALESCE(volume_day4_before.TotalVolume, 0) AS day4_before_volume,
    COALESCE(volume_day3_before.TotalVolume, 0) AS day3_before_volume,
    COALESCE(volume_day2_before.TotalVolume, 0) AS day2_before_volume,
    volume_day_before.TotalVolume AS day_before_volume,
    volume_day_after.TotalVolume AS day_after_volume,
    COALESCE(volume_day2_after.TotalVolume, 0) AS day2_after_volume,
    COALESCE(volume_day3_after.TotalVolume, 0) AS day3_after_volume,
    COALESCE(volume_day4_after.TotalVolume, 0) AS day4_after_volume,
    COALESCE(volume_day5_after.TotalVolume, 0) AS day5_after_volume
FROM
    reg_data
LEFT JOIN volume AS volume_day6_before
    ON reg_data.pit_ticker = volume_day6_before.Symbol 
    AND reg_data.day_6_before = volume_day6_before.Date
LEFT JOIN volume AS volume_day5_before
    ON reg_data.pit_ticker = volume_day5_before.Symbol 
    AND reg_data.day_5_before = volume_day5_before.Date
LEFT JOIN volume AS volume_day4_before
    ON reg_data.pit_ticker = volume_day4_before.Symbol 
    AND reg_data.day_4_before = volume_day4_before.Date
LEFT JOIN volume AS volume_day3_before
    ON reg_data.pit_ticker = volume_day3_before.Symbol 
    AND reg_data.day_3_before = volume_day3_before.Date
LEFT JOIN volume AS volume_day2_before
    ON reg_data.pit_ticker = volume_day2_before.Symbol 
    AND reg_data.day_2_before = volume_day2_before.Date
LEFT JOIN volume AS volume_day_before
    ON reg_data.pit_ticker = volume_day_before.Symbol 
    AND reg_data.day_before = volume_day_before.Date
LEFT JOIN volume AS volume_day_after
    ON reg_data.pit_ticker = volume_day_after.Symbol 
    AND reg_data.day_after = volume_day_after.Date
LEFT JOIN volume AS volume_day2_after
    ON reg_data.pit_ticker = volume_day2_after.Symbol 
    AND reg_data.day_2_after = volume_day2_after.Date
LEFT JOIN volume AS volume_day3_after
    ON reg_data.pit_ticker = volume_day3_after.Symbol 
    AND reg_data.day_3_after = volume_day3_after.Date
LEFT JOIN volume AS volume_day4_after
    ON reg_data.pit_ticker = volume_day4_after.Symbol 
    AND reg_data.day_4_after = volume_day4_after.Date
LEFT JOIN volume AS volume_day5_after
    ON reg_data.pit_ticker = volume_day5_after.Symbol 
    AND reg_data.day_5_after = volume_day5_after.Date
WHERE 
    volume_day_before.TotalVolume IS NOT NULL 
    AND volume_day_after.TotalVolume IS NOT NULL
"""

# Execute the query 
reg_data = con.execute(query).df()

# Close the connection

# Scale shorting by total trading volume
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [20]:
reg_data

,CUSIP,permno,permco,pit_ticker,OFTIC,CNAME,ANNOUNCE,SURPRISE,POS_SURPRISE,pre_open,...,day5_before_volume,day4_before_volume,day3_before_volume,day2_before_volume,day_before_volume,day_after_volume,day2_after_volume,day3_after_volume,day4_after_volume,day5_after_volume
0,73936310,84330,15207,PWAV,PWAV,POWERWAVE TECHNO,2010-10-28 16:05:00,0.010582,True,False,...,176229.0,4987159.0,664917.0,478094.0,1149767.0,2898097.0,1225914.0,1174824.0,784714.0,1317287.0
1,80603710,80362,12933,SCSC,SCSC,SCANSOURCE INC,2010-10-28 16:05:00,0.002179,True,False,...,20156.0,28730.0,30871.0,103222.0,34335.0,134683.0,63442.0,13541.0,7046.0,7647.0
2,20440W10,85602,32143,SID,SID,SID NACIONAL,2010-10-28 18:00:00,-0.013357,False,False,...,1112641.0,1252919.0,1044743.0,1440608.0,953591.0,760193.0,796310.0,1264570.0,1075372.0,1121395.0
3,40049110,79442,29916,SIM,SIM,SIMEC (GPO),2010-10-28 16:25:00,-0.088195,False,False,...,7600.0,6275.0,7377.0,6700.0,85207.0,7300.0,6075.0,6851.0,4950.0,5500.0
4,84546710,63765,4338,SWN,SWN,SOUTHWSTN ENERGY,2010-10-28 17:00:00,0.000306,True,False,...,722492.0,635262.0,1133530.0,1296035.0,972049.0,2367553.0,2818861.0,1289620.0,2037356.0,1220353.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187542,42726X10,90730,46854,HBOS,HBOS,HERITAGE FIN GRO,2011-10-26 16:00:00,0.007456,True,False,...,600.0,1000.0,800.0,1000.0,1600.0,1200.0,1900.0,1000.0,21100.0,10395.0
187543,74727D30,12493,15632,QADA,QADA,QAD INC,2013-11-26 16:15:00,0.000651,True,False,...,910.0,600.0,1790.0,4750.0,5965.0,7791.0,1218.0,12202.0,0.0,4541.0
187544,45172L10,93209,53323,CTC,CTC,IFM INVESTMENTS,2011-05-16 07:00:00,-0.015936,False,True,...,0.0,620.0,14650.0,2300.0,3102.0,4300.0,12451.0,28550.0,27567.0,9829.0
187545,59630420,14932,55027,MBCN,MBCN,MIDDLEFIELD BANC,2017-02-07 16:05:00,-0.002308,False,False,...,0.0,0.0,0.0,0.0,185.0,16.0,0.0,0.0,0.0,501.0


In [21]:
pd.DataFrame(reg_data.columns)

,0
0,CUSIP
1,permno
2,permco
3,pit_ticker
4,OFTIC
...,...
65,day_after_volume
66,day2_after_volume
67,day3_after_volume
68,day4_after_volume


In [22]:
# Daily variable preparation

# Cumulative daily shorting before
for i in range(2, 6):
    cols = ['day_be_shorts'] + [f'day{j}_be_shorts' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_be_shorts'] = reg_data[cols].sum(axis=1) / i

# Cumulative daily shorting after
for i in range(2, 6):
    cols = ['day_af_shorts'] + [f'day{j}_af_shorts' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_af_shorts'] = reg_data[cols].sum(axis=1) / i

# Cumulative off-exchange total volume
for i in range(2, 6):
    cols = ['day_before_volume'] + [f'day{j}_before_volume' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_before_volume'] = (reg_data[cols].sum(axis=1) / i) + 1
    cols = ['day_after_volume'] + [f'day{j}_after_volume' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_after_volume'] = (reg_data[cols].sum(axis=1) / i) + 1

# Cumulative on-exchange total volume
for i in range(2, 6):
    cols = ['day_before_volume_crsp'] + [f'day{j}_before_volume_crsp' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_before_volume_crsp'] = (reg_data[cols].sum(axis=1) / i) + 1
    cols = ['day_after_volume_crsp'] + [f'day{j}_after_volume_crsp' for j in range(2, i + 1)]
    reg_data[f'cumu{i}_after_volume_crsp'] = (reg_data[cols].sum(axis=1) / i) + 1

cols = [f'day{j}_before_volume' for j in range(2, 7)]
reg_data['wk_before_volume'] = (reg_data[cols].sum(axis=1) / 5) + 1
cols = [f'day{j}_before_volume_crsp' for j in range(2, 7)]
reg_data['wk_before_volume_crsp'] = (reg_data[cols].sum(axis=1) / 5) + 1

reg_data['avg_month_before_shorts'] += 1

# Scaled shorts calculations
reg_data['day_be_shorts_sc'] = reg_data['day_be_shorts'] / (reg_data['day_before_volume'] + 1)
reg_data['day_af_shorts_sc'] = reg_data['day_af_shorts'] / (reg_data['day_after_volume'] + 1)
reg_data['day_be_shorts_sc_cr'] = reg_data['day_be_shorts'] / (reg_data['day_before_volume_crsp'] + 1)
reg_data['day_af_shorts_sc_cr'] = reg_data['day_af_shorts'] / (reg_data['day_after_volume_crsp'] + 1)
reg_data['day_be_shorts_sc_wk'] = reg_data['day_be_shorts'] / reg_data['wk_before_volume']
reg_data['day_af_shorts_sc_wk'] = reg_data['day_af_shorts'] / reg_data['wk_before_volume']
reg_data['day_be_shorts_sc_cr_wk'] = reg_data['day_be_shorts'] / reg_data['wk_before_volume_crsp']
reg_data['day_af_shorts_sc_cr_wk'] = reg_data['day_af_shorts'] / reg_data['wk_before_volume_crsp']
reg_data['day_be_shorts_mo'] = reg_data['day_be_shorts'] / reg_data['avg_month_before_shorts']
reg_data['day_af_shorts_mo'] = reg_data['day_af_shorts'] / reg_data['avg_month_before_shorts']

for i in range(2, 6):
    # Off-exchange volume (same window)
    reg_data[f'cumu{i}_be_shorts_sc'] = reg_data[f'cumu{i}_be_shorts'] / reg_data[f'cumu{i}_before_volume']
    reg_data[f'cumu{i}_af_shorts_sc'] = reg_data[f'cumu{i}_af_shorts'] / reg_data[f'cumu{i}_after_volume']
    # On-exchange volume (same window)
    reg_data[f'cumu{i}_be_shorts_sc_cr'] = reg_data[f'cumu{i}_be_shorts'] / reg_data[f'cumu{i}_before_volume_crsp']
    reg_data[f'cumu{i}_af_shorts_sc_cr'] = reg_data[f'cumu{i}_af_shorts'] / reg_data[f'cumu{i}_after_volume_crsp']
    # Off-exchange volume week before
    reg_data[f'cumu{i}_be_shorts_sc_wk'] = reg_data[f'cumu{i}_be_shorts'] / reg_data['wk_before_volume']
    reg_data[f'cumu{i}_af_shorts_sc_wk'] = reg_data[f'cumu{i}_af_shorts'] / reg_data['wk_before_volume']
    # On-exchange volume week before
    reg_data[f'cumu{i}_be_shorts_sc_cr_wk'] = reg_data[f'cumu{i}_be_shorts'] / reg_data['wk_before_volume_crsp']
    reg_data[f'cumu{i}_af_shorts_sc_cr_wk'] = reg_data[f'cumu{i}_af_shorts'] / reg_data['wk_before_volume_crsp']
    # Monthly shorting
    reg_data[f'cumu{i}_be_shorts_mo'] = reg_data[f'cumu{i}_be_shorts'] / reg_data['avg_month_before_shorts']
    reg_data[f'cumu{i}_af_shorts_mo'] = reg_data[f'cumu{i}_af_shorts'] / reg_data['avg_month_before_shorts']

reg_data = reg_data[(reg_data['day_be_shorts_sc'] <= 1) & (reg_data['day_af_shorts_sc'] <= 1)] # Drop shorts/off-exchange total greater than 100%
reg_data = reg_data[(reg_data['cumu2_be_shorts_sc'] <= 1) & (reg_data['cumu2_af_shorts_sc'] <= 1)]
reg_data = reg_data[(reg_data['cumu3_be_shorts_sc'] <= 1) & (reg_data['cumu3_af_shorts_sc'] <= 1)]
reg_data = reg_data[(reg_data['cumu4_be_shorts_sc'] <= 1) & (reg_data['cumu4_af_shorts_sc'] <= 1)]
reg_data = reg_data[(reg_data['cumu5_be_shorts_sc'] <= 1) & (reg_data['cumu5_af_shorts_sc'] <= 1)]

# Prepare for STATA

# Year-quarter time variable
reg_data.loc[:, 'year'] = reg_data['ANNOUNCE'].dt.year
reg_data.loc[:, 'quarter'] = reg_data['ANNOUNCE'].dt.quarter
reg_data.loc[:,'year_quarter'] = (reg_data['year'] - 2010) * 4 + reg_data['quarter']

# Drop duplicates, keeping the later EA
reg_data = reg_data.sort_values(by=['CUSIP', 'year_quarter', 'ANNOUNCE'])
reg_data = reg_data.drop_duplicates(subset=['CUSIP', 'year_quarter'], keep='last')

In [23]:
reg_data.to_csv(working_folder + '/reg_data.txt', index=False)
reg_data.to_stata(working_folder + '/reg_data.dta')

In [99]:
for year in range(2010, 2024):
    file_path = f'{working_folder}/FINRA_30min_Agg/agg_{year}_30min.pkl'

    # Read the data into a dataframe with the same name as the file
    globals()[f'agg_{year}_30min'] = pd.read_pickle(file_path)

In [4]:
# Load in data from each year 2010-2023
# Load aggregate short files for speed
for year in range(2010, 2024):
    file_path = f'{working_folder}/FINRA_1hr_Agg/agg_{year}_1hr.pkl'

    # Read the data into a dataframe with the same name as the file
    globals()[f'agg_{year}_1hr'] = pd.read_pickle(file_path)

In [160]:
# Day before hourly shorts
# Consider dropping rows that are missing for trading hours and only have non trading hours

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('reg_data', reg_data)

# Adjust the query to match on the hour as well
query = f"""
WITH short_data AS (
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2010_1hr UNION ALL
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2011_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2012_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2013_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2014_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2015_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2016_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2017_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2018_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2019_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2020_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2021_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2022_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2023_1hr
)

SELECT 
    reg_data.*, 
    {', '.join([f'COALESCE(short_data_day_before_{i}.Shorts, 0) AS be_shorts_hr{i}' for i in range(24)])},
FROM
    reg_data
LEFT JOIN short_data AS short_data_day_before_{0} ON reg_data.pit_ticker = short_data_day_before_{0}.Symbol AND reg_data.day_before = short_data_day_before_{0}.Date AND short_data_day_before_{0}.Hour = 0
{''.join([f'LEFT JOIN short_data AS short_data_day_before_{i} ON reg_data.pit_ticker = short_data_day_before_{i}.Symbol AND reg_data.day_before = short_data_day_before_{i}.Date AND short_data_day_before_{i}.Hour = {i} ' for i in range(1, 24)])}
ORDER BY 
    reg_data.ANNOUNCE, 
    reg_data.pit_ticker
"""

# Execute the query 
reg_data_hourly = con.execute(query).df()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [161]:
# Day after hourly shorts
# Need to modify so that if shorts are missing for day before and day after, all times, the row is deleted
# Consider dropping rows that are missing for trading hours and only have non trading hours

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('reg_data_hourly', reg_data_hourly)

# Adjust the query to match on the hour as well
query = f"""
WITH short_data AS (
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2010_1hr UNION ALL
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2011_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2012_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2013_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2014_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2015_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2016_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2017_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2018_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2019_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2020_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2021_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2022_1hr UNION ALL 
    SELECT Symbol, Date, Shorts, Hour
    FROM agg_2023_1hr
)

SELECT 
    reg_data_hourly.*, 
    {', '.join([f'COALESCE(short_data_day_after_{i}.Shorts, 0) AS af_shorts_hr{i}' for i in range(24)])},
FROM
    reg_data_hourly
LEFT JOIN short_data AS short_data_day_after_{0} ON reg_data_hourly.pit_ticker = short_data_day_after_{0}.Symbol AND reg_data_hourly.day_before = short_data_day_after_{0}.Date AND short_data_day_after_{0}.Hour = 0
{''.join([f'LEFT JOIN short_data AS short_data_day_after_{i} ON reg_data_hourly.pit_ticker = short_data_day_after_{i}.Symbol AND reg_data_hourly.day_after = short_data_day_after_{i}.Date AND short_data_day_after_{i}.Hour = {i} ' for i in range(1, 24)])}
ORDER BY 
    reg_data_hourly.ANNOUNCE, 
    reg_data_hourly.pit_ticker
"""

# Execute the query 
reg_data_hourly = con.execute(query).df()

# Close the connection
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [162]:
reg_data = pd.read_csv(working_folder + '/reg_data.txt', parse_dates=['ANNOUNCE', 'day_25_before', 'day_6_before', 'day_5_before', 'day_4_before', 'day_3_before', 'day_2_before', 'day_before', 'day_after', 'day_2_after','day_3_after','day_4_after','day_5_after'], date_format='ISO8601')

In [163]:
columns = reg_data_hourly.columns

# Filter columns that start with 'be_shorts_hr' or 'af_shorts_hr'
day_before_columns = [col for col in columns if col.startswith('be_shorts_hr')]
day_after_columns = [col for col in columns if col.startswith('af_shorts_hr')]

select_columns = ', '.join(day_before_columns + day_after_columns)

# Create a connection to an in-memory DuckDB instance
con = duckdb.connect()

con.register('reg_data', reg_data)
con.register('reg_data_hourly', reg_data_hourly)

query = f"""
SELECT 
    reg_data.*, {select_columns}
FROM
    reg_data
LEFT JOIN reg_data_hourly
    ON reg_data.pit_ticker = reg_data_hourly.pit_ticker 
    AND reg_data.ANNOUNCE = reg_data_hourly.ANNOUNCE
"""

# Execute the query 
reg_data_daily_hourly = con.execute(query).df()

# Close the connection

# Scale shorting by total trading volume
con.close()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
columns = reg_data_daily_hourly.columns

# Filter columns that start with 'day_be_shorts_' or 'day_af_shorts_'
columns = [col for col in columns if (col.startswith('be_shorts_hr') | col.startswith('af_shorts_hr'))]

for col in columns:
    if col.startswith('be_shorts_'):
        reg_data_daily_hourly[col+'_sc'] = reg_data_daily_hourly[col] / (reg_data_daily_hourly['day_before_volume'] + 1)
        reg_data_daily_hourly[col+'_sc_cr'] = reg_data_daily_hourly[col] / (reg_data_daily_hourly['day_before_volume_crsp'] + 1)
    if col.startswith('af_shorts_'):
        reg_data_daily_hourly[col+'_sc'] = reg_data_daily_hourly[col] / (reg_data_daily_hourly['day_after_volume'] + 1)
        reg_data_daily_hourly[col+'_sc_cr'] = reg_data_daily_hourly[col] / (reg_data_daily_hourly['day_after_volume_crsp'] + 1)
    reg_data_daily_hourly[col+'_sc_wk'] = reg_data_daily_hourly[col] / reg_data_daily_hourly['wk_before_volume']
    reg_data_daily_hourly[col+'_sc_wk_cr'] = reg_data_daily_hourly[col] / reg_data_daily_hourly['wk_before_volume_crsp']
    reg_data_daily_hourly[col+'_mo'] = reg_data_daily_hourly[col] / reg_data_daily_hourly['avg_month_before_shorts']

reg_data_daily_hourly = reg_data_daily_hourly.copy()

In [165]:
reg_data_daily_hourly.to_csv(working_folder + '/reg_data_daily_hourly.txt', index=False)
reg_data_daily_hourly.to_stata(working_folder + '/reg_data_daily_hourly.dta')

In [166]:
pd.DataFrame(reg_data_daily_hourly.columns)

,0
0,CUSIP
1,permno
2,permco
3,pit_ticker
4,CNAME
...,...
428,af_shorts_hr23_sc
429,af_shorts_hr23_sc_cr
430,af_shorts_hr23_sc_wk
431,af_shorts_hr23_sc_wk_cr


In [167]:
reg_data_daily_hourly

,CUSIP,permno,permco,pit_ticker,CNAME,ANNOUNCE,VALUE,MEDEST,SURPRISE,POS_SURPRISE,...,af_shorts_hr22_sc,af_shorts_hr22_sc_cr,af_shorts_hr22_sc_wk,af_shorts_hr22_sc_wk_cr,af_shorts_hr22_mo,af_shorts_hr23_sc,af_shorts_hr23_sc_cr,af_shorts_hr23_sc_wk,af_shorts_hr23_sc_wk_cr,af_shorts_hr23_mo
0,00030710,14945,55038,AAC,AAC HOLDINGS,2015-04-29 16:05:00,0.004511,0.003008,0.001504,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,00030710,14945,55038,AAC,AAC HOLDG,2015-10-27 17:15:00,0.008562,0.005708,0.002854,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,00030710,14945,55038,AAC,AAC HOLDG,2016-02-23 06:30:00,0.008445,0.008942,-0.000497,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,00030710,14945,55038,AAC,AAC HOLDG,2016-05-05 06:30:00,0.009775,0.007820,0.001955,True,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,00030710,14945,55038,AAC,AAC HOLDG,2016-08-04 06:30:00,0.007653,0.009779,-0.002126,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178252,Y9530810,15857,55491,WVE,WAVE LIFE,2019-03-01 07:30:00,-0.034613,-0.033532,-0.001082,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178253,Y9530810,15857,55491,WVE,WAVE LIFE,2019-05-10 07:30:00,-0.049151,-0.045175,-0.003975,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178254,Y9530810,15857,55491,WVE,WAVE LIFE,2020-05-11 07:30:00,-0.166667,-0.159420,-0.007246,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
178255,Y9530810,15857,55491,WVE,WAVE LIFE,2020-08-10 07:45:00,-0.130979,-0.112756,-0.018223,False,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# STATA Commands

In [24]:
# File paths
input_file = f"{working_folder}/STATA_vars_original.txt"
output_file = f"{working_folder}/STATA_vars_clean.txt"

In [25]:


# File paths
input_file = f"{working_folder}/STATA_vars_original.txt"
output_file = f"{working_folder}/STATA_vars_clean.txt"

# Reading the input file
with open(input_file, 'r') as file:
    lines = file.readlines()

# Preparing the output
output_lines = []
current_command = ""
varlist = []
current_varlist_name = ""

for line in lines:
    line = line.strip()
    if not line:  # Skip empty lines
        continue
    if line.endswith(':'):
        if current_command:
            # Winsorize command
            output_lines.append(f"winsor2 {current_command.strip()}\n")
            # Normalize command
            multiplied_vars = []
            for var in varlist:
                if len(var) >= 29:
                    print(f"{var} is {len(var)} characters\n")
                winsorized_var = f"{var}_w"
                normalized_var = f"{winsorized_var}z"
                multiplied_var = f"{winsorized_var}m"
                output_lines.append(f"egen {normalized_var} = std({winsorized_var})\n")
                output_lines.append(f"gen {multiplied_var} = {normalized_var} / 100\n")
                multiplied_vars.append(multiplied_var)
            # Varlist creation with winsorized and normalized variables
            output_lines.append(f"vl create {current_varlist_name} = ({' '.join(multiplied_vars)})\n")
        current_command = ""
        varlist = []
        current_varlist_name = line[:-1]  # Remove the colon to get the varlist name
    else:
        current_command += line + " "
        varlist.append(line)

if current_command:
    # Winsorize command
    output_lines.append(f"winsor2 {current_command.strip()}\n")
    # Normalize command
    multiplied_vars = []
    for var in varlist:
        if len(var) >= 29:
            output_lines.append(f"{var} is {len(var)} characters\n")
        winsorized_var = f"{var}_w"
        normalized_var = f"{winsorized_var}z"
        multiplied_var = f"{winsorized_var}m"
        output_lines.append(f"egen {normalized_var} = std({winsorized_var})\n")
        output_lines.append(f"gen {multiplied_var} = {normalized_var} / 100\n")
        multiplied_vars.append(multiplied_var)
    # Varlist creation with winsorized and normalized variables
    output_lines.append(f"vl create {current_varlist_name} = ({' '.join(multiplied_vars)})\n")

# Writing to the output file
with open(output_file, 'w') as file:
    file.writelines(output_lines)
